# Libraries

In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
import MDAnalysis.analysis.rms as rms
import matplotlib.pyplot as plt
import glob as g
from tqdm import tqdm
import os
bar_format_str = '{l_bar}{bar:20}{r_bar}{bar:-20b}'

/Users/jhenderson/Software/anaconda3/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib
/Users/jhenderson/Software/anaconda3/lib/python3.11/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


Warnings Ignored: 1.) Atom types missing masses, 2.) Some atom types missing elemental information, 3.) Missing resid information is be reset to one (this is likely water information), 4.) np.int deprecation warning. These errors tend to happen when converting Desmond trajectories to DCD. 

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
systems = ['WT','NPG','ASV','SVD'] 
replicas = [1,2,3,4,5]

This will take a long time to run... Paths have been updated for data availability but calculation has not been rerun.

In [4]:
for sys in systems:
    for rep in replicas:
        print(f'Working on System {sys} Run {rep}.')
        ref_struct = g.glob(f'REF_Structures/{sys}.pdb')[0]
        parm = g.glob(f'../../data/{sys}_{rep}/init.pdb')[0]
        dcd  = g.glob(f'../../data/{sys}_{rep}/converted.dcd')[0]
        u = mda.Universe(parm, dcd, in_memory=True, in_memory_step=1) # Becareful about Changing this. 
        ref = mda.Universe(ref_struct)
        
        # Align the Trajectories
        align.AlignTraj(u, ref, select="name CA",in_memory=True).run()
        
        # Write the trajectory
        protein = u.select_atoms('protein and not name pseu')
        protein.write(f"{sys}-{rep}.pdb")
        with mda.Writer(f"{sys}-{rep}.dcd", protein.n_atoms) as W:
            for ts in tqdm(u.trajectory, bar_format=bar_format_str):
                W.write(protein)
        
        # Do the MDPocket Analysis
        pdb_remake = f'../{sys}-{rep}.pdb'
        dcd_remake = f'../{sys}-{rep}.dcd'
        my_pocket = '../my_pocket.pdb' # This PDB was used to define the EGFR Pocket 
        os.system(f'mkdir my_pocket_{sys}-{rep} 2> null')
        os.system('rm null')
        os.chdir(f'my_pocket_{sys}-{rep}')
        os.system(f'mdpocket --trajectory_file {dcd_remake} --trajectory_format dcd -f {pdb_remake} --selected_pocket {my_pocket}')
        os.chdir('../') 

Working on System WT Run 1.


KeyboardInterrupt: 